In [1]:
import pandas as pd
import nltk
import numpy as np

# Sentiment Analysis
- Unsupervised model
- Supervised model

In [2]:
hotstar_path = 'https://github.com/skathirmani/datasets/raw/master/hotstar.allreviews_Sentiments.csv'
hotstar = pd.read_csv(hotstar_path)
hotstar.head()

,ID,UserName,Created_Date,Reviews,Lower_Case_Reviews,Sentiment_Manual_BP,Sentiment_Manual,Review_Length,DataSource,Year,Month,Date,Sentiment_Polarity
0,1,NaN,8/10/2017,Hh,hh,Negative,Negative,2,Google_PlayStore,2017,8,10,Neutral
1,2,NaN,8/11/2017,No,no,Negative,Negative,2,Google_PlayStore,2017,8,11,Neutral
2,3,asadynwa,8/12/2017,@hotstar_helps during paymnt for premium subsc...,@hotstar_helps during paymnt for premium subsc...,Help,Negative,140,Twitter,2017,8,12,Negative
3,4,jineshroxx,8/11/2017,@hotstartweets I am currently on Jio network a...,@hotstartweets i am currently on jio network a...,Help,Negative,140,Twitter,2017,8,11,Positive
4,5,YaminiSachar,8/5/2017,@hotstartweets the episodes of Sarabhai vs Sar...,@hotstartweets the episodes of sarabhai vs sar...,Help,Negative,140,Twitter,2017,8,5,Neutral


## Supervised Models

### Random Forest

In [3]:
#hotstar.columns
custom_stop_words = []
common_stop_words = nltk.corpus.stopwords.words('english')
stop_words_all = np.hstack([custom_stop_words, common_stop_words])
len(stop_words_all)

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
stemmer = PorterStemmer()

In [4]:
docs = hotstar['Reviews']
docs = docs.str.lower()
docs = docs.str.replace('[^a-z#@ ]', '')
docs = docs.str.split(' ')
words_rows = docs.tolist()
words_all = []
words_rows_clean = []
docs_clean = []
for row in words_rows:
    row_words = [stemmer.stem(word) for word in row if word not in stop_words_all]  
    words_rows_clean.append(row_words)
    docs_clean.append(' '.join(row_words))
    words_all.extend(row_words)

    
model_dtm = CountVectorizer()
sparse_matrix = model_dtm.fit_transform(docs_clean)
dtm = pd.DataFrame(sparse_matrix.toarray(),
                   columns=model_dtm.get_feature_names())
dtm.shape

(5053, 6152)

### Split data

In [5]:
from sklearn.model_selection import train_test_split
train_x, test_x = train_test_split(dtm, test_size=0.2, random_state=0)

train_y = hotstar.iloc[train_x.index]['Sentiment_Manual']
test_y = hotstar.iloc[test_x.index]['Sentiment_Manual']

In [52]:
#test_y.value_counts() / test_y.shape[0] * 100

In [53]:
#train_y.value_counts() / train_y.shape[0] * 100

## Build Model

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
model_rf = RandomForestClassifier()
model_rf.fit(train_x, train_y)

D:\New folder (3)\anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Predict Sentiment

In [8]:
test_y_rf = model_rf.predict(test_x)

### Evaluate Model

In [9]:
(test_y == test_y_rf).sum() / test_y.shape[0] * 100

72.89812067260138

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
senti = SentimentIntensityAnalyzer()
hotstar['sentiment_vader'] = hotstar['Reviews'].apply(lambda v: 
                                                      senti.polarity_scores(v)['compound'])
def assign_sentiment(v):
    if v > 0.25:
        return 'Positive'
    elif v < -0.25:
        return 'Negative'
    else:
        return 'Neutral'
hotstar['sentiment_vader'] = hotstar['sentiment_vader'].apply(assign_sentiment)
test_hotstar = hotstar.iloc[test_y.index]
(test_hotstar['Sentiment_Manual'] == 
 test_hotstar['sentiment_vader']).sum() / test_hotstar.shape[0] * 100

D:\New folder (3)\anaconda\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


55.78635014836796

In [11]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(train_x, train_y)
test_y_pred = model.predict(test_x)
(test_y_pred == test_y).sum() / test_y.shape[0] * 100

68.34817012858556

In [72]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(train_x, train_y)
test_y_pred = model.predict(test_x)
(test_y_pred == test_y).sum() / test_y.shape[0] * 100

62.710187932739856